## Corona-News Scraper from Tagesschau.de

In [1]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

import datetime
import pandas as pd

In [2]:
todays_date = datetime.datetime.now()
date_stamp = str(todays_date.date())

In [3]:
url = 'https://www.tagesschau.de/'
html = urlopen(url)
soup = BeautifulSoup(html, "html5lib")

### Parse Tagesschau.de homepage for new articles

In [4]:
headlines = []
dachzeilen = []
teasers = []
links = []
texts = []
a = soup.find_all('a')
for item in a:
    if item.find('p', class_='dachzeile'):
        links.append(item.get('href'))
        texts.append(item.get_text())

In [5]:
# check the first text / link
print(texts[0])

print(links[0])



Geschäft mit Schutzkleidung
"Jeder versucht, sich zu bereichern"
Im Zuge der Corona-Krise versuchen offenbar zunehmend Firmen, die Not von Kliniken auszunutzen, an Schutzkleidung zu kommen. WDR, NDR und SZ zeigen, wie vor allem mit Atemschutzmasken versucht wird, Geschäfte zu machen.
| mehr


/inland/masken-coronavirus-101.html


### Check which articles on the homepage are about the Corona virus

In [6]:
# corona, covid19, coronavirus
corona_links = []
corona_texts = []
for i in range(len(texts)):
    if ("corona" in texts[i].lower()) | ("covid" in texts[i].lower()):
        if links[i].startswith("/"):
            corona_links.append(links[i])
            corona_texts.append(texts[i])

In [7]:
print(corona_links)
print("")
print(corona_texts)

['/inland/masken-coronavirus-101.html', '/newsticker/liveblog-corona-sonntag-101.html', '/ausland/usa-coronavirus-fauci-103.html', '/ausland/corona-putin-russland-101.html', '/ausland/weltspiegel-singapur-corona-101.html', '/ausland/indien-corona-103.html', '/inland/coronavirus-deutschland-211.html', '/faktenfinder/corona-grippevergleich-101.html', '/ausland/corona-beschraenkungen-europa-103.html', '/ausland/coronavirus-karten-101.html', '/ardimport/regional/hessen/bouffier-thomas-schaefer-101.html']

['\nGeschäft mit Schutzkleidung\n"Jeder versucht, sich zu bereichern"\nIm Zuge der Corona-Krise versuchen offenbar zunehmend Firmen, die Not von Kliniken auszunutzen, an Schutzkleidung zu kommen. WDR, NDR und SZ zeigen, wie vor allem mit Atemschutzmasken versucht wird, Geschäfte zu machen.\n|\xa0mehr\n\n', '\nLiveblog zum Coronavirus \n++ Drei Tote in Wolfsburger Heim ++\nIn einem Wolfsburger Pflegeheim sind drei weitere Männer an Covid19 gestorben. Traurige Zahlen meldet Spanien und in N

### Get full article for each news about corona virus

In [8]:
corona_articles = []
for link in corona_links:
    full_link = "https://www.tagesschau.de" + link
    print(full_link)
    html = urlopen(full_link)
    sub_soup = BeautifulSoup(html, "html5lib")
    article = sub_soup.find_all('p', class_='text small')
    paragraphs = []
    for i in range(len(article)):
        paragraphs.append(article[i].get_text())
    # unifying teh paragraphs
    final_article = " ".join(paragraphs)
    corona_articles.append(final_article)
    

https://www.tagesschau.de/inland/masken-coronavirus-101.html
https://www.tagesschau.de/newsticker/liveblog-corona-sonntag-101.html
https://www.tagesschau.de/ausland/usa-coronavirus-fauci-103.html
https://www.tagesschau.de/ausland/corona-putin-russland-101.html
https://www.tagesschau.de/ausland/weltspiegel-singapur-corona-101.html
https://www.tagesschau.de/ausland/indien-corona-103.html
https://www.tagesschau.de/inland/coronavirus-deutschland-211.html
https://www.tagesschau.de/faktenfinder/corona-grippevergleich-101.html
https://www.tagesschau.de/ausland/corona-beschraenkungen-europa-103.html
https://www.tagesschau.de/ausland/coronavirus-karten-101.html
https://www.tagesschau.de/ardimport/regional/hessen/bouffier-thomas-schaefer-101.html


In [9]:
print(corona_articles)

['Im Zuge der Corona-Krise versuchen offenbar zunehmend Firmen, die Not von Kliniken auszunutzen, an Schutzkleidung zu kommen. WDR, NDR und SZ zeigen, wie vor allem mit Atemschutzmasken versucht wird, Geschäfte zu machen.  Auf dem Markt der medizinischen Schutzausrüstung herrscht Chaos. Seit dem Ausbruch der Corona-Pandemie versucht das Personal in Heilberufen verzweifelt, Atemschutzmasken und Kittel zu besorgen, die benötigt werden, damit sich Mitarbeiter und Patienten bei der Behandlung vor einer Ansteckung mit dem SARS-CoV-2 Virus schützen.  Doch der Preis ist in die Höhe geschnellt, bereits bestellte Ware wird nicht geliefert, das verfügbare Material ist häufig mangelhaft, neue - oft unseriöse - Anbieter versuchen, von der Krise zu profitieren. "Hier muss tatsächlich der Staat einspringen. Das ist nichts, was der Markt auch nur im Ansatz lösen könnte", sagt der SPD-Politiker Karl Lauterbach. Die jetzt vom Bundesgesundheitsministerium vorgeschlagenen Maßnahmen hält er für unzureiche

In [10]:
len(corona_articles)

11

### Make into CSV file

In [11]:
columns = ["date", "url", "article", "resort"]
df = pd.DataFrame(columns=columns)
df["url"] = corona_links
df["article"] = corona_articles
df["date"] = todays_date
df["resort"] = df["url"].apply(lambda x: x.split("/")[1])

In [12]:
df.head()

,date,url,article,resort
0,2020-03-29 18:27:02.611348,/inland/masken-coronavirus-101.html,Im Zuge der Corona-Krise versuchen offenbar zu...,inland
1,2020-03-29 18:27:02.611348,/newsticker/liveblog-corona-sonntag-101.html,In einem Wolfsburger Pflegeheim sind drei weit...,newsticker
2,2020-03-29 18:27:02.611348,/ausland/usa-coronavirus-fauci-103.html,Fast 125.000 Menschen sind in den USA bereits ...,ausland
3,2020-03-29 18:27:02.611348,/ausland/corona-putin-russland-101.html,Gegen die Ausbreitung des Coronavirus hat Russ...,ausland
4,2020-03-29 18:27:02.611348,/ausland/weltspiegel-singapur-corona-101.html,In Singapur bleibt die Zahl der Corona-Infizie...,ausland


### Export CSV

In [13]:
df.to_csv("data/" + date_stamp)